In [35]:
import pandas as pd
import ast
import numpy as np
from matplotlib import pyplot as plt

from sklearn.decomposition import TruncatedSVD, PCA

In [36]:
df = pd.read_json('./Dataset/dataset.json')
df.head(5)
df = df.sample(frac=1).reset_index(drop=True)
df = df.iloc[:1000]

In [37]:
def pca_explnation(features, n_c):
    pca = PCA(n_components=n_c, random_state=42)
    pca.fit(features)
    return pca.explained_variance_ratio_.sum()

def pca_transform(features, n_c):
    pca = PCA(n_components=n_c, random_state=42)
    pca.fit(features)
    return pca.transform(features)

In [38]:
inps = np.concatenate([np.array(i) for i in df['input'].values], axis=0)
outs = np.concatenate([np.array(i) for i in df['output'].values], axis=0)
imgs = np.concatenate([inps, outs], axis=0)
print(imgs.shape)

n, h, w = imgs.shape
imgs_flat = imgs.reshape(n, h*w)

(8000, 15, 15)


In [39]:
# x = range(1, 225)
# y = [pca_explnation(imgs_flat, i) for i in x]

# plt.figure(figsize=(10, 5))
# plt.plot(x, y)
# plt.show()

In [40]:
# # test number of components
# pca = PCA(n_components=10, random_state=42)
# pca.fit(imgs_flat)

# reduced = pca.transform(imgs_flat)
# inverse = pca.inverse_transform(reduced)
# inverse = inverse.reshape(n, h, w)

# plt.imshow(imgs[0].reshape(h, w))
# plt.show()
# plt.imshow(inverse[0] >= 0.5)
# plt.show()

In [41]:
n_c = 60
reduced = pca_transform(imgs_flat, n_c)
print(reduced.shape)
reduced_inps = reduced[:len(inps)].tolist()
reduced_inps = [reduced_inps[i:i+4] for i in range(0, len(reduced_inps), 4)]
reduced_outs = reduced[len(inps):]
reduced_outs = [reduced_outs[i:i+4] for i in range(0, len(reduced_outs), 4)]

df_reduced = df.copy()
df_reduced['input_reduced'] = reduced_inps
df_reduced['output_reduced'] = reduced_outs

print(np.array(df_reduced['input_reduced'][0]).shape)
df_reduced.head(2)

(8000, 60)
(4, 60)


,input,output,operation,kernel,input_reduced,output_reduced
0,"[[[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, ...","[[0.4915625203242748, 2.0279707626235397, 1.48...","[[2.6852835244963753, 2.1976331945982595, 0.03..."
1,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 1, 1, 1, 1]","[[0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, ...","[[-1.2119002941639456, -1.5753306375244485, 1....","[[-0.37237829160499153, -0.5023079152015293, 2..."


In [42]:
df_reduced.to_json(f'./Dataset/dataset1k_reduced_{n_c}.json')